In [2]:
import pandas as pd
# %%
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
#from lightgbm import LGBMClassifier
#from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score, confusion_matrix, recall_score, precision_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.svm import SVC
import imblearn
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [62]:
import numpy as np
from numpy import genfromtxt
import pandas as pd
import json

In [4]:
#Read train dataset
train_df = pd.read_csv("train_dataset.csv")

In [5]:
#Read test dataset
test_df = pd.read_csv("test_dataset.csv")

In [6]:
train_df.drop(["Unnamed: 0"],axis = 1,inplace=True)

In [7]:
test_df.drop(["Unnamed: 0"], axis =1 , inplace=True)

In [8]:
train_df

,CLIENT-sender,COMPANY-sender,JPMC-CLIENT-self,JPMC-CLIENT-sender,JPMC-COMPANY-sender,CLIENT-bene,COMPANY-bene,JPMC-CLIENT-bene,JPMC-COMPANY-bene,EL-SALVADOR-sender,...,other-bene,DEPOSIT-CHECK,EXCHANGE,MAKE-PAYMENT,MOVE-FUNDS,PAY-CHECK,QUICK-PAYMENT,WITHDRAWAL,USD_amount,Label
0,0,0,0,1,0,1,0,0,0,0,...,1,0,0,0,0,0,1,0,581.34,0
1,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,2000.00,0
2,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,613.75,0
3,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,293.82,0
4,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,715.15,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003773,0,0,1,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,105.18,0
1003774,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,104.39,0
1003775,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,823.29,0
1003776,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,427.54,0


In [9]:
#split x and y column
X_train = train_df.loc[:,train_df.columns!='Label']
X_test =test_df.loc[:,test_df.columns!='Label']
y_train = train_df.Label
y_test =  test_df.Label
 

In [13]:
#Upsampling datasets

sm = SMOTE(random_state = 42)
balance_x_train, balance_y_train = sm.fit_resample(X_train,y_train)

In [11]:
balance_x_test, balance_y_test = sm.fit_resample(X_test,y_test)

In [19]:
#Standarization USD_amount feature 
x_train_stand = balance_x_train.copy()
x_test_stand = balance_x_test.copy()
from sklearn.preprocessing import StandardScaler
#fit on training data column
scale = StandardScaler().fit(balance_x_train[['USD_amount']])
#transform the training data column
x_train_stand['USD_amount']= scale.transform(x_train_stand[['USD_amount']])
#transform the testing data column
x_test_stand["USD_amount"] = scale.transform(x_test_stand[['USD_amount']])


In [33]:
def print_performance_metric(actual_y,predict_y):
    accuracy_on_test_dataset = accuracy_score(actual_y, predict_y)
    f1_score_test_dataset = f1_score(actual_y, predict_y, average='macro',zero_division=1)
    recall_score_test_dataset = recall_score(actual_y, predict_y, average = 'macro', zero_division=1)
    precision_score_test_dataset = precision_score(actual_y, predict_y, average ='macro', zero_division=1)
    confu_matrix = confusion_matrix(actual_y, predict_y)
    classify_report = classification_report(actual_y, predict_y)
    print('Accuracy score on the test dataset: ',accuracy_on_test_dataset)
    print('Recall score on the test dataset:', recall_score_test_dataset)
    print('Precision score on the test dataset:', precision_score_test_dataset)
    print('F1 score on the test dataset:', f1_score_test_dataset)
    print(confu_matrix)
    print(classify_report)
    return accuracy_on_test_dataset, f1_score_test_dataset,recall_score_test_dataset,precision_score_test_dataset
    

In [34]:
def model_score(name, model,x_train,x_test,y_train,y_test):
    cv = StratifiedKFold(n_splits = 5, random_state = 42, shuffle = True)
    scores = []
    f1_scores = []
    oversampler = SMOTE(random_state=42)
    for train_fold_index, val_fold_index in cv.split(x_train,y_train):
        x_train_fold = x_train.iloc[train_fold_index]
        y_train_fold = y_train[train_fold_index]
        x_val_fold = x_train.iloc[val_fold_index]
        y_val_fold = y_train[val_fold_index]

        #upsample on the training dataset
        X_train_fold_upsample, y_train_fold_upsample = oversampler.fit_resample(x_train_fold,y_train_fold)
        model.fit(X_train_fold_upsample,y_train_fold_upsample)
        y_predict = model.predict(x_val_fold)
        score = accuracy_score(y_val_fold,y_predict)
        scores.append(score)
        f1 = f1_score(y_val_fold, y_predict,  average = 'macro', zero_division=1)
        f1_scores.append(f1)
        
    #validate on the test dataset
    y_test_predict = model.predict(x_test)
    accuracy_test,f1_score_test,recall_score_test,precision_score_test = print_performance_metric(y_test,y_test_predict)
    
    
    report_score = {'macro_f1_folds':np.array(f1_scores), 'accuracy_test':accuracy_test,'macro_f1_test':f1_score_test,
                   'recall_score_test':recall_score_test,'precision_score_test':precision_score_test}
    return report_score

In [35]:
models = []
models.append(('LR', LogisticRegression(solver='lbfgs', max_iter=700)))
models.append(('RF', RandomForestClassifier()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('DT', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(decision_function_shape='ovo', probability=True)))

In [59]:
all_model_reports = {}

In [43]:
#run the Logistic Regression model
logit_report = model_score(models[0][0], models[0][1],x_train_stand, x_test_stand,balance_y_train,balance_y_test)

Accuracy score on the test dataset:  0.780115014918901
Recall score on the test dataset: 0.7801150149189011
Precision score on the test dataset: 0.7816245665734138
F1 score on the test dataset: 0.7798199650764338
[[395526  88759]
 [124215 360070]]
              precision    recall  f1-score   support

           0       0.76      0.82      0.79    484285
           1       0.80      0.74      0.77    484285

    accuracy                           0.78    968570
   macro avg       0.78      0.78      0.78    968570
weighted avg       0.78      0.78      0.78    968570



In [55]:
logit_report["running_time_(min)"] = '794m 32.2s'
logit_report["running_time_(hours)"] = "13 hours"
logit_report

In [60]:
all_model_reports['LR'] = logit_report
all_model_reports['LR']['macro_f1_folds'] = list(all_model_reports['LR']['macro_f1_folds'])
all_model_reports

In [63]:
out_file = open("ML_report.json","a")
json.dump(all_model_reports,out_file)
out_file.close()

In [69]:
#run the RandomForest model
random_report = model_score(models[1][0], models[1][1],x_train_stand, x_test_stand,balance_y_train,balance_y_test)

Accuracy score on the test dataset:  0.7800737169228863
Recall score on the test dataset: 0.7800737169228863
Precision score on the test dataset: 0.7891196204038259
F1 score on the test dataset: 0.7783399053094213
[[420609  63676]
 [149338 334947]]
              precision    recall  f1-score   support

           0       0.74      0.87      0.80    484285
           1       0.84      0.69      0.76    484285

    accuracy                           0.78    968570
   macro avg       0.79      0.78      0.78    968570
weighted avg       0.79      0.78      0.78    968570



In [70]:
random_report["running_time_(min)"] = "847m 19.4s"
random_report["running_time_(hours)"] = "14 hours"
random_report

{'macro_f1_folds': array([0.86070416, 0.86220558, 0.86174839, 0.86107692, 0.86075964]),
 'accuracy_test': 0.7800737169228863,
 'macro_f1_test': 0.7783399053094213,
 'recall_score_test': 0.7800737169228863,
 'precision_score_test': 0.7891196204038259,
 'running_time_(min)': '847m 19.4s',
 'running_time_(hours)': '14 hours'}

In [71]:
all_model_reports['RF'] = random_report
all_model_reports['RF']['macro_f1_folds'] = list(all_model_reports['RF']['macro_f1_folds'])

out_file = open("ML_report.json","a")
json.dump(all_model_reports,out_file)
out_file.close()

In [72]:
knn_report = model_score(models[2][0], models[2][1],x_train_stand, x_test_stand,balance_y_train,balance_y_test)

Accuracy score on the test dataset:  0.7697440556696986
Recall score on the test dataset: 0.7697440556696986
Precision score on the test dataset: 0.7758628064881903
F1 score on the test dataset: 0.7684601432914915
[[408838  75447]
 [147572 336713]]
              precision    recall  f1-score   support

           0       0.73      0.84      0.79    484285
           1       0.82      0.70      0.75    484285

    accuracy                           0.77    968570
   macro avg       0.78      0.77      0.77    968570
weighted avg       0.78      0.77      0.77    968570



In [73]:
knn_report["running_time_(min)"] = "2323m 5.2s"
knn_report["running_time_(hours)"] = "38 hours"
knn_report

{'macro_f1_folds': array([0.86528463, 0.86563509, 0.86561551, 0.86516811, 0.86628385]),
 'accuracy_test': 0.7697440556696986,
 'macro_f1_test': 0.7684601432914915,
 'recall_score_test': 0.7697440556696986,
 'precision_score_test': 0.7758628064881903,
 'running_time_(min)': '2323m 5.2s',
 'running_time_(hours)': '38 hours'}

In [86]:
all_model_reports['KNN'] = knn_report
all_model_reports['KNN']['macro_f1_folds'] = list(all_model_reports['KNN']['macro_f1_folds'])

out_file = open("ML_report.json","w")
json.dump(all_model_reports,out_file, indent=6)
out_file.close()

In [85]:
all_model_reports

{'LR': {'macro_f1_folds': [0.7318459243545,
   0.7322881679489202,
   0.7327312842935103,
   0.7319083729454767,
   0.7324630123825426],
  'accuracy_test': 0.780115014918901,
  'macro_f1_test': 0.7798199650764338,
  'recall_score_test': 0.7801150149189011,
  'precision_score_test': 0.7816245665734138,
  'running_time_(min)': '794m 32.2s',
  'running_time_(hours)': '13 hours'},
 'RF': {'macro_f1_folds': [0.8607041580744229,
   0.8622055824249245,
   0.8617483904295209,
   0.8610769169693608,
   0.8607596398482289],
  'accuracy_test': 0.7800737169228863,
  'macro_f1_test': 0.7783399053094213,
  'recall_score_test': 0.7800737169228863,
  'precision_score_test': 0.7891196204038259,
  'running_time_(min)': '847m 19.4s',
  'running_time_(hours)': '14 hours'},
 'KNN': {'macro_f1_folds': [0.8652846250360733,
   0.8656350932578165,
   0.8656155135552277,
   0.8651681094817174,
   0.8662838518860122],
  'accuracy_test': 0.7697440556696986,
  'macro_f1_test': 0.7684601432914915,
  'recall_score_t